In [1]:
import gensim
import jieba
from gensim.models import word2vec
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [2]:
news = pd.read_csv('news.csv',encoding='gbk')
news.head()

,content,title,channel
0,北京时间7月21日消息，2022年男篮亚洲杯在雅加达继续进行，新西兰队在1/4决赛中88-7...,亚洲杯-韩国队两人被驱逐不敌新西兰 止步八强,体育
1,北京时间7月22日消息，CBA山东队官方宣布，与外援泰勒-罗切斯特完成签约。罗切斯特出生于美...,山东队官宣签下罗切斯特 上赛季场均23+4+6,体育
2,7月20日，张可盈晒出一组夏日清爽写真，照片中张可盈一身纯白色抹胸连衣裙，长发披肩，发丝随风...,张可盈夏日清爽写真 甜美少女气息与何幸运反差极大,明星
3,新浪娱乐讯 21日，柳岩最新一组时尚大片曝光。大片中，她身穿一袭黑纱裙，搭配黑色直发在月光下...,柳岩月下起舞大片氛围拉满 穿一袭黑纱裙尽显轻盈古典美,明星
4,新浪娱乐讯 据韩国媒体报道，男子组合NCT将出演时隔2年再次回归的《偶像运动会》。此前，NC...,《偶像运动会》时隔两年再办 男团NCT将参演竞技,音乐


In [3]:
news['channel'].value_counts()

体育    2
明星    2
音乐    1
Name: channel, dtype: int64

In [4]:
label_transfer = {
    '体育':0,
    '明星':1,
    '音乐':2
}
news['label'] = news['channel'].apply(lambda x:label_transfer[x])
news.head()

,content,title,channel,label
0,北京时间7月21日消息，2022年男篮亚洲杯在雅加达继续进行，新西兰队在1/4决赛中88-7...,亚洲杯-韩国队两人被驱逐不敌新西兰 止步八强,体育,0
1,北京时间7月22日消息，CBA山东队官方宣布，与外援泰勒-罗切斯特完成签约。罗切斯特出生于美...,山东队官宣签下罗切斯特 上赛季场均23+4+6,体育,0
2,7月20日，张可盈晒出一组夏日清爽写真，照片中张可盈一身纯白色抹胸连衣裙，长发披肩，发丝随风...,张可盈夏日清爽写真 甜美少女气息与何幸运反差极大,明星,1
3,新浪娱乐讯 21日，柳岩最新一组时尚大片曝光。大片中，她身穿一袭黑纱裙，搭配黑色直发在月光下...,柳岩月下起舞大片氛围拉满 穿一袭黑纱裙尽显轻盈古典美,明星,1
4,新浪娱乐讯 据韩国媒体报道，男子组合NCT将出演时隔2年再次回归的《偶像运动会》。此前，NC...,《偶像运动会》时隔两年再办 男团NCT将参演竞技,音乐,2


In [5]:
'/'.join(jieba.cut(news['content'][0],cut_all=False))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\FANGFA~1.SHA\AppData\Local\Temp\jieba.cache
Loading model cost 0.725 seconds.
Prefix dict has been built successfully.


'北京/时间/7/月/21/日/消息/，/2022/年/男篮/亚洲杯/在/雅加达/继续/进行/，/新西兰队/在/1///4/决赛/中/88/-/78/击败/韩国队/，/晋级/四强/。/韩国队/后卫线/两员/大将/许勋/和/许雄/分别/因/伤病/和/新冠/缺阵/，/另外/两名/主将/李/大成/和/崔俊用/在/比赛/中均/累计/两次/技术犯规/被/罚下场/。/罗/健儿/为/韩国队/砍/下/19/分/14/板/3/封盖/，/金钟/奎/13/分/3/板/，/崔俊用/11/分/9/板/7/助攻/，/李/大成/11/分/，/姜相载/7/分/。/新西兰队/的/卡梅隆/得到/22/分/9/板/8/助攻/，/蒂明斯/6/分/14/板/，/米尔纳/14/分/5/板/，/法伦/索恩/16/分/7/板/，/亨特/8/分/4/板/，/穆雷/9/分/6/板/。/根据/赛程/，/新西兰队/将/在/半决赛/对阵/澳大利亚队/。'

In [6]:
news['分词'] = news['content'].apply(lambda x: [i for i in jieba.cut(x,cut_all=False)])
news.head()

,content,title,channel,label,分词
0,北京时间7月21日消息，2022年男篮亚洲杯在雅加达继续进行，新西兰队在1/4决赛中88-7...,亚洲杯-韩国队两人被驱逐不敌新西兰 止步八强,体育,0,"[北京, 时间, 7, 月, 21, 日, 消息, ，, 2022, 年, 男篮, 亚洲杯,..."
1,北京时间7月22日消息，CBA山东队官方宣布，与外援泰勒-罗切斯特完成签约。罗切斯特出生于美...,山东队官宣签下罗切斯特 上赛季场均23+4+6,体育,0,"[北京, 时间, 7, 月, 22, 日, 消息, ，, CBA, 山东队, 官方, 宣布,..."
2,7月20日，张可盈晒出一组夏日清爽写真，照片中张可盈一身纯白色抹胸连衣裙，长发披肩，发丝随风...,张可盈夏日清爽写真 甜美少女气息与何幸运反差极大,明星,1,"[7, 月, 20, 日, ，, 张可盈, 晒, 出, 一组, 夏日, 清爽, 写真, ，,..."
3,新浪娱乐讯 21日，柳岩最新一组时尚大片曝光。大片中，她身穿一袭黑纱裙，搭配黑色直发在月光下...,柳岩月下起舞大片氛围拉满 穿一袭黑纱裙尽显轻盈古典美,明星,1,"[新浪, 娱乐, 讯, , 21, 日, ，, 柳岩, 最新, 一组, 时尚, 大片, 曝..."
4,新浪娱乐讯 据韩国媒体报道，男子组合NCT将出演时隔2年再次回归的《偶像运动会》。此前，NC...,《偶像运动会》时隔两年再办 男团NCT将参演竞技,音乐,2,"[新浪, 娱乐, 讯, , 据, 韩国, 媒体报道, ，, 男子, 组合, NCT, 将,..."


In [7]:
f = open('text1.txt','w',encoding='utf-8')
for i in range(news.shape[0]):
    f.writelines(' '.join(news['分词'][i])+'\n')
f.close()

In [8]:
sentences = word2vec.Text8Corpus('text1.txt')
sentences

In [9]:
model = Word2Vec(sentences, min_count=1)
model.save('words_models.model')

In [10]:
model1 = Word2Vec.load('words_models.model')
model1

In [11]:
word_vectors=model1.wv
word_vectors

In [12]:
def pad(x, maxlengh=500):
    x1 = np.zeros((maxlengh, 100))
    if len(x) == 0:
        return x1
    if len(x) <= maxlengh:
        x1[: len(x)] = x
    else:
        x1 = x[: maxlengh]
    x1 = np.array(x1).reshape(maxlengh, 100)
    return x1

def apply_text_trainsfer(text):
#     text = [text.strip()]
    vec = [word_vectors[w] for w in text if w in word_vectors]
    vec = pad(vec)
    return vec

In [13]:
news.head()

,content,title,channel,label,分词
0,北京时间7月21日消息，2022年男篮亚洲杯在雅加达继续进行，新西兰队在1/4决赛中88-7...,亚洲杯-韩国队两人被驱逐不敌新西兰 止步八强,体育,0,"[北京, 时间, 7, 月, 21, 日, 消息, ，, 2022, 年, 男篮, 亚洲杯,..."
1,北京时间7月22日消息，CBA山东队官方宣布，与外援泰勒-罗切斯特完成签约。罗切斯特出生于美...,山东队官宣签下罗切斯特 上赛季场均23+4+6,体育,0,"[北京, 时间, 7, 月, 22, 日, 消息, ，, CBA, 山东队, 官方, 宣布,..."
2,7月20日，张可盈晒出一组夏日清爽写真，照片中张可盈一身纯白色抹胸连衣裙，长发披肩，发丝随风...,张可盈夏日清爽写真 甜美少女气息与何幸运反差极大,明星,1,"[7, 月, 20, 日, ，, 张可盈, 晒, 出, 一组, 夏日, 清爽, 写真, ，,..."
3,新浪娱乐讯 21日，柳岩最新一组时尚大片曝光。大片中，她身穿一袭黑纱裙，搭配黑色直发在月光下...,柳岩月下起舞大片氛围拉满 穿一袭黑纱裙尽显轻盈古典美,明星,1,"[新浪, 娱乐, 讯, , 21, 日, ，, 柳岩, 最新, 一组, 时尚, 大片, 曝..."
4,新浪娱乐讯 据韩国媒体报道，男子组合NCT将出演时隔2年再次回归的《偶像运动会》。此前，NC...,《偶像运动会》时隔两年再办 男团NCT将参演竞技,音乐,2,"[新浪, 娱乐, 讯, , 据, 韩国, 媒体报道, ，, 男子, 组合, NCT, 将,..."


In [14]:
news['fea_vec'] = news['分词'].apply(apply_text_trainsfer)
news.head()

,content,title,channel,label,分词,fea_vec
0,北京时间7月21日消息，2022年男篮亚洲杯在雅加达继续进行，新西兰队在1/4决赛中88-7...,亚洲杯-韩国队两人被驱逐不敌新西兰 止步八强,体育,0,"[北京, 时间, 7, 月, 21, 日, 消息, ，, 2022, 年, 男篮, 亚洲杯,...","[[-0.006511525716632605, 0.009026700630784035,..."
1,北京时间7月22日消息，CBA山东队官方宣布，与外援泰勒-罗切斯特完成签约。罗切斯特出生于美...,山东队官宣签下罗切斯特 上赛季场均23+4+6,体育,0,"[北京, 时间, 7, 月, 22, 日, 消息, ，, CBA, 山东队, 官方, 宣布,...","[[-0.006511525716632605, 0.009026700630784035,..."
2,7月20日，张可盈晒出一组夏日清爽写真，照片中张可盈一身纯白色抹胸连衣裙，长发披肩，发丝随风...,张可盈夏日清爽写真 甜美少女气息与何幸运反差极大,明星,1,"[7, 月, 20, 日, ，, 张可盈, 晒, 出, 一组, 夏日, 清爽, 写真, ，,...","[[0.008123216219246387, -0.004369305446743965,..."
3,新浪娱乐讯 21日，柳岩最新一组时尚大片曝光。大片中，她身穿一袭黑纱裙，搭配黑色直发在月光下...,柳岩月下起舞大片氛围拉满 穿一袭黑纱裙尽显轻盈古典美,明星,1,"[新浪, 娱乐, 讯, , 21, 日, ，, 柳岩, 最新, 一组, 时尚, 大片, 曝...","[[0.002210981212556362, -0.007390774320811033,..."
4,新浪娱乐讯 据韩国媒体报道，男子组合NCT将出演时隔2年再次回归的《偶像运动会》。此前，NC...,《偶像运动会》时隔两年再办 男团NCT将参演竞技,音乐,2,"[新浪, 娱乐, 讯, , 据, 韩国, 媒体报道, ，, 男子, 组合, NCT, 将,...","[[0.002210981212556362, -0.007390774320811033,..."


In [15]:
X_train_data = []
for i in range(news.shape[0]):
    X_train_data.append(apply_text_trainsfer(news['分词'][i]))

In [16]:
X_train_data[0]

array([[-0.00651153,  0.0090267 , -0.00155046, ..., -0.00996138,
         0.00702593, -0.00738297],
       [-0.00719306,  0.00642535, -0.00195216, ..., -0.00300673,
        -0.00578624,  0.00371005],
       [ 0.00812322, -0.00436931, -0.00106676, ..., -0.00576125,
        -0.00163452,  0.00557623],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
X_train_data = np.array(X_train_data)
X_train_data.shape

(5, 500, 100)

In [19]:
y_train_onehot = keras.utils.to_categorical(news['label'].tolist())
y_train_onehot

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [20]:
y_train_onehot.shape

(5, 3)

In [21]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

def create_model(X_train, nclass=3):
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.1))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(nclass, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [22]:
model2 = create_model(X_train_data, nclass=3)
model2

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 20)                2580      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 119,891
Trainable params: 119,891
Non-trainable params: 0
_________________________________________________________________


In [23]:
model2.fit(X_train_data, y_train_onehot, epochs=10, batch_size=2, verbose=2)

Epoch 1/10
3/3 - 2s - loss: 1.1001 - accuracy: 0.4000 - 2s/epoch - 733ms/step
Epoch 2/10
3/3 - 1s - loss: 1.0984 - accuracy: 0.4000 - 560ms/epoch - 187ms/step
Epoch 3/10
3/3 - 1s - loss: 1.0965 - accuracy: 0.4000 - 513ms/epoch - 171ms/step
Epoch 4/10
3/3 - 1s - loss: 1.0937 - accuracy: 0.4000 - 533ms/epoch - 178ms/step
Epoch 5/10
3/3 - 1s - loss: 1.0899 - accuracy: 0.4000 - 523ms/epoch - 174ms/step
Epoch 6/10
3/3 - 1s - loss: 1.0904 - accuracy: 0.4000 - 537ms/epoch - 179ms/step
Epoch 7/10
3/3 - 1s - loss: 1.0869 - accuracy: 0.4000 - 524ms/epoch - 175ms/step
Epoch 8/10
3/3 - 1s - loss: 1.0900 - accuracy: 0.4000 - 538ms/epoch - 179ms/step
Epoch 9/10
3/3 - 1s - loss: 1.0750 - accuracy: 0.4000 - 526ms/epoch - 175ms/step
Epoch 10/10
3/3 - 1s - loss: 1.0815 - accuracy: 0.4000 - 523ms/epoch - 174ms/step


In [24]:
model2.predict(X_train_data)

array([[0.3357901 , 0.37771147, 0.28649843],
       [0.3357901 , 0.37771145, 0.28649846],
       [0.3357901 , 0.37771145, 0.28649846],
       [0.3357901 , 0.37771145, 0.28649846],
       [0.3357901 , 0.37771147, 0.28649843]], dtype=float32)

In [25]:
y_train_onehot

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)